In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt
import Fvvsc

## 1. Create function for calculating entropy density
- inputs: array for energy values (q) and two density matrices for neutrinos and anti neutrinos
- the function uses trapezoid rule to calculate the entropy density integral: 
    - for each density matrix calculate the eigenvalues
    - use eigenvalues to calculate tr[$\rho$ ln $\rho$]
- use data file to calculate s at every time step

$s = -\int_{0}^{\infty} q^2 \,dq \, \mathrm{trc} [\rho_q\mathrm{ln}\rho_q]$

In [2]:
## my trapezoid rule from dm

#@nb.jit(nopython=True)
def trapezoid(x,y): # x is array of indep vars and y is array of integrand values
    N=len(x)
    summ = 0
    for i in range(N-1):
        I = 0.5*(y[i]+ y[i+1])*(x[i+1] - x[i])
        summ = summ + I
    return summ

In [ ]:
old_data = np.load('Run35.npz', allow_pickle = True)
data_noscat = np.load('1000keV-run_long.npz', allow_pickle = True)

#NEW DATA FILE
data = np.load('1000keV-run_long-Collision.npz', allow_pickle = True)

In [37]:
#array for energy values
settings = data['settings'].item()
T = settings['T']
N = settings['N'] + 1 #=201
emax = settings['eps_max']
q_vals = np.linspace(0, emax*T, N)

#Matrices for rho and rho bar
rawdata = data['raw'].item()
nu3D = rawdata['nu3D']
nubar3D = rawdata['nubar3D']

nu3D.shape[1]
q_vals.shape[0]
rawdata.keys()

dict_keys(['initial array', 'time', 'Ht', 'dt', 'dN', 'nu3D', 'nubar3D', 'nu_init matrix', 'nu_bar_init matrix'])

In [25]:
def solve_entropy(q_vals, rho, rhobar): #rho and rhobar are 2 dimensional
    #initialize
    N = q_vals.shape[0] # this is now 201, starting at zero
    traces = [0]
    tracesbar = [0]

    for i in range(N-1): # for every matrix in each time step
        # get matrix into components
        P0, Px, Py, Pz = rho[i,:] 
    
        # calculate eigenvalues
        p_mag = np.sqrt(Px**2 + Py**2 + Pz**2)
        lambda1 = 0.5*P0 + 0.5*P0*p_mag
        lambda2 = 0.5*P0 - 0.5*P0*p_mag
        klnk1 = lambda1*np.log(lambda1) if lambda1 > 0 else 0
        klnk2 = lambda2*np.log(lambda2) if lambda2 > 0 else 0
        trace = (klnk1 + klnk2) * q_vals[i]**2
        # or q_vals[i+1]**2?????
    
        #now do it all again for nubar
        P0bar, Pxbar, Pybar, Pzbar = rhobar[i,:]
        pbar_mag = np.sqrt(Pxbar**2 + Pybar**2 + Pzbar**2)
        lambda1bar = 0.5*P0bar + 0.5*P0bar*pbar_mag
        lambda2bar = 0.5*P0bar - 0.5*P0bar*pbar_mag
        klnk1bar = lambda1bar*np.log(lambda1bar) if lambda1bar > 0 else 0
        klnk2bar = lambda2bar*np.log(lambda2bar) if lambda2bar > 0 else 0
        tracebar = (klnk1bar + klnk2bar) * q_vals[i]**2
#NOTE: for this to work with my trap function, trace and tracebar need to be arrays, so at the end of every loop here
# i should save the trace and tracebar values into an array and then i could put those into trap
        #ok so after each calculation of trace and tracebar, ill append them to a list, then i can put those lits into trapezoid instead
        traces.append(trace)
        tracesbar.append(tracebar)
    
        trace_vals = np.array(traces)
        tracebar_vals = np.array(tracesbar)

        #calculate s
    s = -1/(2*np.pi**2)*(trapezoid(q_vals[:len(trace_vals)], trace_vals) + trapezoid(q_vals[:len(trace_vals)], tracebar_vals))
    return s

## 2. Use data file to calculate s

In [26]:
s_try1 = solve_entropy(q_vals, nu3D[0,:,:], nubar3D[0,:,:])
s_try2 = solve_entropy(q_vals, nu3D[1,:,:], nubar3D[1,:,:])
s_try3 = solve_entropy(q_vals, nu3D[2,:,:], nubar3D[2,:,:])

print(s_try1)
print(s_try2)
print(s_try3)

1.1375950834129156
1.1375950834129096
1.1375950834129027


In [27]:
s = np.zeros((N)) 

for j in range(N):
    s[j] = np.array([solve_entropy(q_vals, nu3D[j,:,:], nubar3D[j,:,:])])
    # do this in function
    print(s[j])

1.1375950834129156
1.1375950834129096
1.1375950834129027
1.1375950834128956
1.137595083412888
1.1375950834128807
1.1375950834128756
1.1375950834128714
1.1375950834128663
1.1375950834128614
1.1375950834128556
1.1375950834128497
1.1375950834128437
1.137595083412836
1.1375950834128274
1.137595083412817
1.1375950834128066
1.1375950834127948
1.1375950834127837
1.1375950834127702
1.1375950834127557
1.1375950834127408
1.1375950834127246
1.1375950834127064
1.1375950834126871
1.1375950834126647
1.1375950834126425
1.1375950834126187
1.137595083412591
1.1375950834125632
1.1375950834125317
1.1375950834124995
1.137595083412465
1.1375950834124238
1.1375950834123845
1.1375950834123423
1.1375950834122932
1.1375950834122408
1.1375950834121893
1.1375950834121347
1.1375950834120738
1.1375950834120034
1.1375950834119304
1.1375950834118533
1.1375950834117745
1.137595083411695
1.1375950834116138
1.1375950834115205
1.1375950834114148
1.1375950834112967
1.1375950834111694
1.1375950834110349
1.1375950834108937

1.1375950830609456
1.137595083060932
1.1375950830609172
1.1375950830609065
1.1375950830608963
1.1375950830608885
1.137595083060881
1.1375950830608728
1.137595083060866
1.13759508306086
1.1375950830608537
1.1375950830608477
1.1375950830608401
1.1375950830608335
1.1375950830608266
1.13759508306082
1.1375950830608121
1.1375950830608053
1.1375950830607986
1.1375950830607908
1.1375950830607842
1.137595083060777
1.1375950830607708
1.137595083060764
1.137595083060757
1.1375950830607502
1.1375950830607433
1.1375950830607362
1.1375950830607293
1.1375950830607224
1.1375950830607142
1.137595083060704
1.1375950830606933
1.13759508306068
1.137595083060666
1.1375950830606512
1.1375950830606363
1.1375950830606185
1.1375950830605999
1.137595083060581
1.1375950830605595
1.1375950830605361
1.1375950830605104
1.1375950830604835
1.137595083060454
1.1375950830604216
1.137595083060386
1.1375950830603492
1.13759508306031
1.1375950830602668
1.1375950830602237
1.1375950830601729
1.1375950830601194
1.1375950830

## 3. Calculate derivative $\frac{1}{s}\frac{ds}{dt} = \frac{d \mathrm{ln}s}{dt}$ approximate as $\approx \frac{\mathrm{ln}s(t + dt) - \mathrm{ln}s(t)}{dt}$
- to calculate $s(t + dt)$ need to evolve $(P_0, \vec{P})$ from $t$ to $t+dt$ for each of our density matrices

In [53]:
data = np.load('1000keV-run_long-Collision.npz', allow_pickle = True)
data.files
data['coll_nu']

array([[[ 3.78288585e-26,  0.00000000e+00,  0.00000000e+00,
          3.02096355e-27],
        [ 7.38069616e-26,  0.00000000e+00,  0.00000000e+00,
          5.99279818e-27],
        [ 1.11715225e-25,  0.00000000e+00,  0.00000000e+00,
          9.21843394e-27],
        ...,
        [-2.51082307e-26,  0.00000000e+00,  0.00000000e+00,
         -3.23684024e-27],
        [-2.55222149e-26,  0.00000000e+00,  0.00000000e+00,
         -3.28974853e-27],
        [-2.45697016e-26,  0.00000000e+00,  0.00000000e+00,
         -3.16688593e-27]],

       [[ 3.78640941e-26, -1.59093110e-26,  1.07026471e-26,
          3.69933144e-27],
        [ 7.38072117e-26,  5.74791679e-28,  3.88911019e-27,
          6.02007185e-27],
        [ 1.11727890e-25, -1.83508804e-26,  6.60608786e-27,
          9.49340777e-27],
        ...,
        [-2.51082305e-26, -5.14518125e-30,  2.12371104e-29,
         -3.23676730e-27],
        [-2.55222147e-26, -5.06607782e-30,  2.15390566e-29,
         -3.28967478e-27],
        [-2.456

In [59]:
C_vals = data['coll_nu']
C_bar_vals = data['coll_nubar']

In [66]:
def dsdt(q_vals, rho, rho_bar, col_nu, col_nubar): #rho and rhobar are 2 dimensional
    #initialize
    N = q_vals.shape[0] # this is now 201, starting at zero
    dtraces = [0]
    dtraces_bar = [0]

    for i in range(N-1): # for every matrix in each time step
        # get matrix into components
        P0, Px, Py, Pz = rho[i,:]
        C0, Cx, Cy, Cz = col_nu[i,:]
        pmag = np.sqrt(Px**2 + Py**2 + Pz**2)
        lambda1 = 0.5*P0 + 0.5*P0*pmag
        lambda2 = 0.5*P0 - 0.5*P0*pmag
        
     #new derivaite stuff
        dP0dt = 2*C0
        dpmagdt = -2*(C0/P0)*pmag**2 + (4/P0)*(Cx*Px + Cy*Py + Cz*Pz)

        dk1dt = 0.5*dP0dt*(1+pmag) + 0.5*P0*dpmagdt
        dk2dt = 0.5*dP0dt*(1-pmag) - 0.5*P0*dpmagdt
        
        dklnk1 = dk1dt*(np.log(lambda1)+1) if lambda1 > 0 else 0
        dklnk2 = dk2dt*(np.log(lambda2)+1) if lambda2 > 0 else 0
        dtrace = (dklnk1 + dklnk2) * q_vals[i]**2
        # or q_vals[i+1]**2?????
    
        #now do it all again for nubar
        P0_bar, Px_bar, Py_bar, Pz_bar = rho_bar[i,:]
        C0_bar, Cx_bar, Cy_bar, Cz_bar = col_nubar[i,:]
        pmag_bar = np.sqrt(Px_bar**2 + Py_bar**2 + Pz_bar**2)
        lambda1_bar = 0.5*P0_bar + 0.5*P0_bar*pmag_bar
        lambda2_bar = 0.5*P0_bar - 0.5*P0_bar*pmag_bar
        
        #new derivaite stuff
        dP0dt_bar = 2*C0_bar
        dpmagdt_bar = -2*(C0_bar/P0_bar)*pmag_bar**2 + (4/P0_bar)*(Cx_bar*Px_bar + Cy_bar*Py_bar + Cz_bar*Pz_bar)

        dk1dt_bar = 0.5*dP0dt_bar*(1+pmag_bar) + 0.5*P0*dpmagdt_bar
        dk2dt_bar = 0.5*dP0dt_bar*(1-pmag_bar) - 0.5*P0*dpmagdt_bar
        
        dklnk1_bar = dk1dt_bar*(np.log(lambda1_bar)+1) if lambda1_bar > 0 else 0
        dklnk2_bar = dk2dt_bar*(np.log(lambda2_bar)+1) if lambda2_bar > 0 else 0      
        dtrace_bar = (dklnk1_bar + dklnk2_bar) * q_vals[i]**2
#NOTE: for this to work with my trap function, trace and tracebar need to be arrays, so at the end of every loop here
# i should save the trace and tracebar values into an array and then i could put those into trap
        #ok so after each calculation of trace and tracebar, ill append them to a list, then i can put those lits into trapezoid instead
        dtraces.append(dtrace)
        dtraces_bar.append(dtrace_bar)
    
        dtrace_vals = np.array(dtraces)
        dtrace_vals_bar = np.array(dtraces_bar)

        #calculate s
    s = -1/(2*np.pi**2)*(trapezoid(q_vals[:len(dtrace_vals)], dtrace_vals) + trapezoid(q_vals[:len(dtrace_vals)], dtrace_vals_bar))
    return s

In [67]:
ds = np.zeros((N)) 

for j in range(N):
    ds[j] = np.array([dsdt(q_vals, nu3D[j,:,:], nubar3D[j,:,:], C_vals[j,:,:], C_bar_vals[j,:,:])])
    # do this in function
    print(ds[j])

-1.2008790305994192e-23
-1.2008790165308997e-23
-1.200879014471631e-23
-1.2008790156256373e-23
-1.200879014591613e-23
-1.2008790148170413e-23
-1.200879014760622e-23
-1.2008790138292565e-23
-1.2008790150337575e-23
-1.2008790137319444e-23
-1.2008790139286476e-23
-1.2008790162468575e-23
-1.2008790141471352e-23
-1.2008790128288698e-23
-1.2008790146798135e-23
-1.2008790152125066e-23
-1.2008790134367916e-23
-1.200879015383674e-23
-1.2008790128841989e-23
-1.2008790157673268e-23
-1.2008790146788818e-23
-1.200879016538381e-23
-1.2008790157779408e-23
-1.200879016376093e-23
-1.2008790158373376e-23
-1.2008790133579254e-23
-1.2008790139211185e-23
-1.2008790151720494e-23
-1.2008790165188326e-23
-1.200879014554236e-23
-1.2008790134292211e-23
-1.200879016290335e-23
-1.2008790153652765e-23
-1.2008790156423078e-23
-1.2008790155362667e-23
-1.2008790158382453e-23
-1.2008790134906227e-23
-1.2008790149363841e-23
-1.2008790157092664e-23
-1.200879014231057e-23
-1.2008790151067185e-23
-1.2008790159609034e-23
-